# Setting Response Metadata
API cells can have an optional companion response cell to set response metadata (e.g. status, headers, etc.). An example comment, in python, for a companion cell would be `# ResponseInfo GET /resource`. The aforementioned companion cell would run after the corresponding API cell for `# GET /resource` is run. The companion cell relays response metadata to the kernel gateway by printing a well known json structure to standard out. An example of the json structure is as follows:

```
{
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 201
}
```

Currently, `headers` and `status` are the only values supported. `headers` should be an object of key-value pairs of header names to header values for the response. `status` should be an integer value for the response status code.

## Setup
The cells below install dependencies and creates some basic code functionality for our API.

In [ ]:
!pip install dicttoxml

In [ ]:
import json
from dicttoxml import dicttoxml
import hashlib

In [ ]:
PERSON_INDEX = {
    1 : {
        'id' : 1,
        'name' : 'Corey',
        'age' : 26,
        'location' : 'Austin, TX'
    }
}

In [ ]:
def get_person(person_id):
    person_id = int(person_id)
    return PERSON_INDEX[person_id]

def get_person_hash(person_id):
    hash_value = hashlib.md5()
    person = get_person(person_id)
    hash_value.update("{}-{}-{}-{}".format(
            person['id'],
            person['name'],
            person['age'],
            person['location']
    ).encode('UTF-8'))
    return hash_value.hexdigest()

def serialize_person(person, content_type):
    if content_type == 'application/json':
        return json.dumps(person)
    elif content_type == 'application/xml' or content_type == 'text/xml':
        return dicttoxml(person).decode('UTF-8')
    elif content_type == 'text/html':
        return '''<p>{} is {} years old and lives in {}.</p>'''.format(
            person['name'], person['age'], person['location']
        )
    
def get_request_content_type(request):
    if 'headers' in request and 'Content-Type' in request['headers']:
        return request['headers']['Content-Type']
    else:
        return 'text/html'

def get_request_etag(request):
    if 'headers' in request and 'If-None-Match' in request['headers']:
        return request['headers']['If-None-Match']
    else:
        return None

In [ ]:
REQUEST = json.dumps({
    'path' : {
        'id' : '1'
    }, 
    'headers' : {
        'Content-Type' : 'application/json',
        'If-None-Match' : 'e958b9efafbd6429bfad0985df27a1fb'
    }
})

## The API Endpoint Cell
This cell is responsible for creating the endpoint response value.

In [ ]:
# GET /person/:id
request = json.loads(REQUEST)
etag = get_request_etag(request)
person_id = int(request['path']['id'])
current_person_hash = get_person_hash(person_id)
status_code = 200
if etag == current_person_hash:
    status_code = 304
else:        
    person = get_person(person_id)
    response_value = serialize_person(person, get_request_content_type(request))
    print(response_value)

## Setting The Response Metadata
This cell sets the response metadata by printing a json string to standard out.

In [ ]:
# ResponseInfo GET /person/:id
print(json.dumps({
        'headers' : {
            'Content-Type' : get_request_content_type(request),
            'ETag' : current_person_hash
        },
        'status' : status_code
    })
)